In [1]:
# !pip3 install sacrebleu

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()

In [4]:
import malaya

model = malaya.translation.ms_en.transformer(model = 'noisy-base')

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

Downloading:   0%|          | 0.00/233M [00:00<?, ?B/s]

2022-07-07 11:53:28.061463: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-07 11:53:28.064901: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-07 11:53:28.064918: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-07-07 11:53:28.064922: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2022-07-07 11:53:28.064959: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was unable to find libcuda.so DSO loaded into this program
2022-07-07 11:53:28.064978: I

In [6]:
with open('ms-en-right.test') as fopen:
    right = fopen.read().split('\n')
    
with open('ms-en-left.test') as fopen:
    left = fopen.read().split('\n')

In [7]:
from tqdm import tqdm

batch_size = 16

results = []
for i in tqdm(range(0, len(left), batch_size)):
    r_greedy = model.greedy_decoder(left[i:i + batch_size])
    results.extend(r_greedy)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 5475/5475 [1:19:43<00:00,  1.14it/s]


In [8]:
len(results)

87596

In [9]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(r)
        filtered_right.append(right[no])

In [10]:
refs = [filtered_right]
sys = filtered_left

In [11]:
r = bleu.corpus_score(sys, refs)

That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.


In [12]:
r.__dict__

{'name': 'BLEU',
 'score': 71.72549311084798,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '86.3/74.8/67.2/61.0 (BP = 1.000 ratio = 1.002 hyp_len = 2004164 ref_len = 2001100)',
 'bp': 1.0,
 'counts': [1729673, 1434493, 1228551, 1062705],
 'totals': [2004164, 1916569, 1829125, 1742100],
 'sys_len': 2004164,
 'ref_len': 2001100,
 'precisions': [86.303965144569,
  74.84692698254015,
  67.16604934053167,
  61.00137764766661],
 'prec_str': '86.3/74.8/67.2/61.0',
 'ratio': 1.0015311578631751}